# Ejemplo ANOVA

**Autor:** Jazna Meza Hidalgo

**Correo Electrónico:** ja.meza@profesor.duoc.cl

**Fecha de Creación:** Septiembre de 2024  
**Versión:** 1.0  

# Definición

ANOVA (Análisis de Varianza) es una técnica estadística que se utiliza para comparar las medias de más de dos grupos para determinar si al menos una de las medias es significativamente diferente de las otras




---

## Problemas con el uso de la Tarjeta Compra Más (TCM)

Usted, como *Data Scientist* se desempeña en una empresa del Retail donde, como experto, tiene como labor determinar las áreas débiles y fuertes a identificar y proponer mejoras.

En una reunión (tormenta de ideas) se determina que el uso de nuestra tarjeta presenta áreas de mejora en varios aspectos. Sin embargo, es necesario verificar o refutar las apreciaciones que, durante la reunión, se plantearon sobre la TCM. Estás se pueden indexar como sigue:

* **Montos**: El comité de promociones discute que ellos han focalizado apropiadamente las ofertas, de tal
forma que han logrado incrementar los montos de compras mensuales, y para comprobar indican que en 2021 fue de \$1.150.000 en promedio, asegurando que en 2022 debe ser superior.
* **Antigüedad**: El área de fidelización de clientes (renegociación) es acusada de impedir que antiguos clientes incrementen sus compras al limitar sus niveles de endeudamiento. En otras palabras, entre los que usan la tarjeta, los clientes antiguos tiene montos medios \$100.000 menor que los clientes nuevos (las tarjetas de los clientes identificados cón números menores a 250.000 corresponden a clientes antiguos).


Su trabajo será, mediante análisis de datos mediante tablas, gráficos y test de hipótesis, estudiar los puntos anteriores. Para el cumplimiento de lo anterior, el Departamento de Informática les hace llegar una pequeña muestra aleatoria simple de clientes con información histórica y los atributos solicitados (ver `TCM2022.xlsx`). Las variables de la muestra son las siguientes:

* `Cliente`: código de identificación del cliente – asociada a la fecha de otorgamiento.
* `Edad`: en años actualizada a diciembre 2022.
* `Sexo`: F (femenino), M (masculino).
* `Region`: Indica si el cliente pertenece a la región metropolitana (RM) o no (No RM).
* `Uso2022`: 1 si ha usado la tarjeta durante 2022, 0 si solo la ha usado en 2021.
* `Monto2022`: Monto total en compras desde enero a diciembre de 2022 (en miles).

In [ ]:
import pandas as pd
import numpy as np
import math
import random
import scipy.stats as stats

import matplotlib.pyplot as plt
import seaborn as sns

from ipywidgets import interactive
from scipy.stats import norm

In [ ]:
!wget https://raw.githubusercontent.com/JaznaLaProfe/Fundamentos-de-Machine-Learning/main/data/TCM2022.xlsx

--2024-09-10 22:52:39--  https://raw.githubusercontent.com/JaznaLaProfe/Fundamentos-de-Machine-Learning/main/data/TCM2022.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17783 (17K) [application/octet-stream]
Saving to: ‘TCM2022.xlsx’

TCM2022.xlsx        100%[===================>]  17.37K  --.-KB/s    in 0s      

2024-09-10 22:52:40 (116 MB/s) - ‘TCM2022.xlsx’ saved [17783/17783]



In [ ]:
# Carga de la data
data = pd.read_excel("TCM2022.xlsx")

In [ ]:
# Cantidad de observaciones y caracteristicas disponibles
print("Hay {0} observaciones que cuentan con {1} columnas".format(data.shape[0], data.shape[1]))

Hay 450 observaciones que cuentan con 6 columnas


## Tabla de la media del Monto de 2022 según Región


In [ ]:
# Obtiene el promedio de cada grupo
data.groupby("Region")["Monto2022"].agg(["mean"]).round(3)

,mean
Region,
No RM,708.649
RM,698.837


## Definición de hipótesis

 * $H_0$ = La media de consumo en el año 2022 de los clientes residentes en comunas distintas a la RM es igual a la media de consumo de los clientes residentes en la RM.

 * $H_1$ = La media de consumo en el año 2022 de los clientes residentes en comunas distintas a la RM es distinta a la media de consumo de los clientes residentes en la RM.


In [ ]:

# Muestra aleatoria de 100 clientes
unique_customer_id = list(data['Cliente'].unique())
# Establece una semilla para extraer siempre la misma muestra
random.seed(30)
sample_student_id = random.sample(unique_customer_id, 100)
sample_df = data[data['Cliente'].isin(sample_student_id)].reset_index(drop=True)

# Dos variables de interes
sample_df = sample_df[['Region', 'Monto2022']]
groups = sample_df.groupby('Region').count().reset_index()
# Visualiza
groups

,Region,Monto2022
0,No RM,56
1,RM,44


## Aplicación test ANOVA

In [ ]:
# Crea la tabla principal de ANOVA
data_table = [['Between Groups', '', '', '', '', '', ''], ['Within Groups', '', '', '', '', '', ''], ['Total', '', '', '', '', '', '']]
anova_table = pd.DataFrame(data_table, columns = ['Source of Variation', 'SS', 'df', 'MS', 'F', 'P-value', 'F crit'])
anova_table.set_index('Source of Variation', inplace = True)

# Calcula ENTRE los grupos
x_bar = sample_df['Monto2022'].mean()
SSTR = sample_df.groupby('Region').count() * (sample_df.groupby('Region').mean() - x_bar)**2
anova_table['SS']['Between Groups'] = SSTR['Monto2022'].sum()

# Calcula EN los grupos
SSE = (sample_df.groupby('Region').count() - 1) * sample_df.groupby('Region').std()**2
anova_table['SS']['Within Groups'] = SSE['Monto2022'].sum()

# Calcula el total
SSTR = SSTR['Monto2022'].sum() + SSE['Monto2022'].sum()
anova_table['SS']['Total'] = SSTR

# Actualiza los grados de libertad
anova_table['df']['Between Groups'] = sample_df['Region'].nunique() - 1
anova_table['df']['Within Groups'] = sample_df.shape[0] - sample_df['Region'].nunique()
anova_table['df']['Total'] = sample_df.shape[0] - 1

# Calcula la media de los cuadrados
anova_table['MS'] = anova_table['SS'] / anova_table['df']

# Calcula F
F = anova_table['MS']['Between Groups'] / anova_table['MS']['Within Groups']
anova_table['F']['Between Groups'] = F

# p-value
anova_table['P-value']['Between Groups'] = 1 - stats.f.cdf(F, anova_table['df']['Between Groups'], anova_table['df']['Within Groups'])

# F critico
alpha = 0.05
#
tail_hypothesis_type = "two-tailed"
if tail_hypothesis_type == "two-tailed":
    alpha /= 2
anova_table['F crit']['Between Groups'] = stats.f.ppf(1-alpha,
                                                      anova_table['df']['Between Groups'],
                                                      anova_table['df']['Within Groups'])

# Mostrar tabla ANOVA
anova_table.round(2)

,SS,df,MS,F,P-value,F crit
Source of Variation,,,,,,
Between Groups,61156.800065,1,61156.800065,0.10376,0.748049,5.181823
Within Groups,57761587.439935,98,589403.953469,,,
Total,57822744.24,99,584068.123636,,,


In [ ]:
# Resultado final
print("\n","-"*80)
print("El enfoque del valor crítico para la prueba de hipótesis en la regla de decisión")
conclusion = "No se pudo rechazar la hipótesis nula."
if anova_table['F']['Between Groups'] > anova_table['F crit']['Between Groups']:
    conclusion = "Se rechaza la hipótesis nula."
print("F-score es: {0:.2f} y valor critico es: {1:.2f}".format(anova_table['F']['Between Groups'],
                                                               anova_table['F crit']['Between Groups']))
print(conclusion)


 --------------------------------------------------------------------------------
El enfoque del valor crítico para la prueba de hipótesis en la regla de decisión
F-score es: 0.10 y valor critico es: 5.18
No se pudo rechazar la hipótesis nula.


# Conclusión:

No hay evidencia suficiente en los datos para concluir que la media de consumo de los clientes residentes en comunas distintas a la Región Metropolitana (RM) sea diferente de la media de consumo de los clientes residentes en la RM durante el año 2022, con un 5% de nivel de significancia.


# Cálculos en la tabla ANOVA

Una tabla ANOVA se compone de varias filas y columnas que muestran diferentes estadísticas que permiten entender si las diferencias entre las medias de los grupos son estadísticamente significativas. Los cálculos que se incluyen en la tabla son:



## 1. Suma de Cuadrados (Sum of Squares - SS)


Es una medida de variación que cuantifica la dispersión de los datos.



### Suma de Cuadrados Entre Grupos (SSB o SSA):

Calcula la variabilidad debido a la interacción entre los grupos.

Fórmula:
$𝑆𝑆_{entre}=\sum_{i=1}^{n}n_{i}(\bar{y}_{i}-\bar{y})^2$

donde:

$n_{i}$ = número de observaciones en el grupo 𝑖

$\bar{y}_{i}$= media del grupo 𝑖

$\bar{y}$= media global



### Suma de Cuadrados Dentro de los Grupos (SSE o SSW):

Calcula la variabilidad dentro de cada grupo.

Fórmula:
$SS_{dentro}=\sum_{i=1}^{k}\sum_{j=1}^{n_{i}}(\bar{y_{ij}}-\bar{y})^2$

 donde:

$y_{ij}$ = j-ésima observación en el grupo 𝑖



### Suma de Cuadrados Total (SST):

Es la suma de las variaciones entre y dentro de los grupos.

$SS_{total}=SS_{entre} + SS_{dentro}$



## 2. Grados de Libertad (Degrees of Freedom - df)
Los grados de libertad son el número de valores que son "libres" de variar cuando se estima un parámetro estadístico:





### Grados de Libertad Entre Grupos ($df_{entre})$:

Calculado como el número de grupos menos uno:
$df_{entre}$ = k−1, donde

𝑘 es el número de grupos.



### Grados de Libertad Dentro de los Grupos ($df_{dentro}$)

Calculado como el número total de observaciones menos el número de grupos:
$df_{dentro}$ = 𝑁 − 𝑘 donde

N es el número total de observaciones.



### Grados de Libertad Total ($df_{total}$):

Es la suma de los grados de libertad entre y dentro de los grupos:
$𝑑𝑓_{total}$ = 𝑁 −1

## 3. Media Cuadrática (Mean Square - MS)
La Media Cuadrática se calcula dividiendo la Suma de Cuadrados por sus respectivos grados de libertad:



### Media Cuadrática Entre Grupos ($MS_{entre}$):

Fórmula: $𝑀𝑆_{entre} = \frac{𝑆𝑆_{entre}}{df_{entre}}$



### Media Cuadrática Dentro de los Grupos ($MS_{dentro}$):

Fórmula: $𝑀𝑆_{dentro} = \frac{𝑆𝑆_{dentro}}{df_{dentro}}$



## 4. Estadístico F (F-statistic)

El estadístico F se utiliza para comparar la variabilidad entre los grupos con la variabilidad dentro de los grupos. Se calcula como:

$F = \frac{M𝑆_{entre}}{MS_{dentro}}$

Este valor se compara con un valor crítico de una distribución F con los grados de libertad correspondientes para determinar si se rechaza la hipótesis nula.



## 5. Valor P (P-value)
El valor P es la probabilidad de obtener un valor F al menos tan extremo como el observado, dado que la hipótesis nula es verdadera. Si el valor P es menor que el nivel de significancia (
𝛼, comúnmente 0.05), se rechaza la hipótesis nula.

# Significado de "Two-Tailed" (Bilateral)

  

1. **Contexto de Hipótesis Estadísticas**:

   - En una prueba de hipótesis, se tienen dos hipótesis: la hipótesis nula $H_0$ y la hipótesis alternativa $H_1$.  

   - La **hipótesis nula** generalmente sugiere que no hay efecto o diferencia (por ejemplo, "la media es igual a 0").

   - La **hipótesis alternativa** sugiere que sí hay un efecto o diferencia (por ejemplo, "la media no es igual a 0").

  

2. **Prueba Bilateral (Two-Tailed)**:

   - Una **prueba bilateral** se utiliza cuando la hipótesis alternativa $H_1$ es que el parámetro de interés **no es igual** a un valor específico (no mayor ni menor, sino simplemente diferente). Por ejemplo:

     $H_0: \mu = \mu_0 \quad \text{vs.} \quad H_1: \mu \neq \mu_0$

   - Esta prueba evalúa tanto la posibilidad de que el parámetro sea **mayor** como **menor** que el valor hipotético. Por eso se le llama "two-tailed" (dos colas).